# Модель движения машин по кругу

In [6]:
from tkinter import *
import time
import random
import numpy as np
import math

In [7]:
class car():    
    def nearest_cars(self):
        n_h = None
        n_t = None 
        p_h = None 
        p_t = None
        l_next_h = 10000
        l_prev_h = -10000
        l_next_t = 10000
        l_prev_t = -10000
        for car_ in self.cars_list:
            if (self.line == car_.line):
                if car_.x - self.x < 0:
                    if car_.x - self.x > l_prev_h:
                        l_prev_h = car_.x - self.x
                        p_h = car_
                if car_.x - self.x > 0:
                    if car_.x - self.x < l_next_h:
                        l_next_h = car_.x - self.x
                        n_h = car_
            else:
                if car_.x - self.x < 0:
                    if car_.x - self.x > l_prev_t:
                        l_prev_t = car_.x - self.x
                        p_t = car_
                if car_.x - self.x >= 0:
                    if car_.x - self.x < l_next_t:
                        l_next_t = car_.x - self.x
                        n_t = car_
        # теперь краевые случаи
        if n_h == None:
            x_tmp = self.x
            for car_ in self.cars_list:
                if (car_.x < x_tmp) & (car_.line == self.line):
                    x_tmp = car_.x
                    n_h = car_
        if n_t == None:
            x_tmp = self.x
            for car_ in self.cars_list:
                if (car_.x < x_tmp) & (car_.line != self.line):
                    x_tmp = car_.x
                    n_t = car_
        if p_h == None:
            x_tmp = self.x
            for car_ in self.cars_list:
                if (car_.x > x_tmp) & (car_.line == self.line):
                    x_tmp = car_.x
                    p_h = car_
        if p_t == None:
            x_tmp = self.x
            for car_ in self.cars_list:
                if (car_.x > x_tmp) & (car_.line != self.line):
                    x_tmp = car_.x
                    p_t = car_
            
        result = [n_h, n_t, p_h, p_t]
        return(result)
    
    def __init__(self, canvas, x0, v0, line, cars_list):
        self.canvas = canvas
        self.size = [20, 10]
        self.v = v0
        self.x = x0
        self.line = line
        self.a = 0
        self.cars_list = cars_list
        n_cars = self.nearest_cars()
        self.next_here = None # ссылка на следующуу машину в этом ряду
        self.next_there = None # ссылка на следующую машину в соседнем ряду
        self.prev_here = None 
        self.prev_there = None
        #self.l = self.next_here.x - self.x
        
        if line == 1:
            self.y = 240
        else:
            self.y = 260
        self.id = canvas.create_rectangle(self.x - self.size[0]/2 + 50,
                                          self.y - self.size[1]/2,
                                          self.x + self.size[0]/2 + 50,
                                          self.y + self.size[1]/2,
                                          fill = "red")
        
    
    
    def obgon(self):
        if self.line == 1:
            self.line = 2
            dy = 20
            self.y = 260
        else:
            self.line = 1
            dy = -20
            self.y = 240
        self.canvas.move(self.id, 0, dy)
        
        for car_ in self.cars_list:
            n_cars = car_.nearest_cars()
            car_.next_here = n_cars[0] 
            car_.next_there = n_cars[1]
            car_.prev_here = n_cars[2] 
            car_.prev_there = n_cars[3]

        # если здесь искать ближайшие машины для всего массива, то не будет проблемы со слипшимися тачками
        # СДЕЛАНО, ПРОБЛЕМА ОСТАЛАСЬ
#         n_cars = self.nearest_cars() 
#         self.next_here = n_cars[0] 
#         self.next_there = n_cars[1]
#         self.prev_here = n_cars[2] 
#         self.prev_there = n_cars[3]
        
        
    def draw(self):
        #---------------1-------------
        n_cars = self.nearest_cars()
        self.next_here = n_cars[0] 
        self.next_there = n_cars[1]
        self.prev_here = n_cars[2] 
        self.prev_there = n_cars[3]
        
        self_l = self.next_here.x - self.x
        #--------------2-------------
        dx = self.v * t + (self.a * (t**2))/2
        old_x = self.x
        self.x = self.x + dx
        #--------------3------------
        self.l = self.next_here.x - self.x
        if self.l <= l_min + 10:
            if (self.next_there.x - self.x > l_min*2) & (self.x - self.prev_there.x > l_min*2):
                self.obgon()
        
        la = self.l - l_min - 40 # здесь задается расстояние для рассчета ускорения
        anew = la * alpha
        vnew = self.v + self.a * t
        if vnew < 0:
            vnew = 0
        if (vnew > v_max):
            vnew = v_max
            anew = 0
        if (self.l < l_min):
            vnew = self.next_here.v
            self.l = l_min
            
        self.v = vnew
        self.a = anew
        #-------------4--------------
        delta = (self.x % 1800) - old_x % 1800
        self.canvas.move(self.id, delta, 0)
        

In [8]:
def draw_cars(v_max, l_min, t, alpha):
    tk = Tk()
    tk.title('graph')
    tk.resizable(0, 0)
    canvas = Canvas(tk, width=1900, height=500, highlightthickness=0)
    canvas.pack()
    
    canvas.create_line(0, 250, 1900, 250)
    canvas.create_line(0, 230, 1900, 230)
    canvas.create_line(0, 270, 1900, 270)
    tk.update()
    
    # задать массив скоростей
    arv = []
    for i in range(30):
        arv.append(150)
        
    # задать массив координат
    ar = []
    for i in range(30):
        ar.append((1800/30) * i)
        ar = sorted(ar)
      
    # задать массив полосы
    lines = []
    for i in range(30):
        lines.append(random.randint(1, 2))
        
    cars = []
    for i in range(30):
        car_ = car(canvas, ar[i], arv[i], lines[i], cars)
        cars.append(car_)
    for car_ in cars:
        n_cars = car_.nearest_cars()
        car_.next_here = n_cars[0] # ссылка на следующуу машину в этом ряду
        car_.next_there = n_cars[1] # ссылка на следующую машину в соседнем ряду
        car_.prev_here = n_cars[2] 
        car_.prev_there = n_cars[3]
        
    while 1:
        for car_ in cars:
            car_.draw()
        tk.update_idletasks()
        tk.update()
        time.sleep(0.05)

In [9]:
v_max = 200 # максимальная скорость
l_min = 20 + 20 # мин расстояние до след. машины
t = 0.1 # время опроса
alpha = 0.3 # коэффициент ускорения

In [10]:
draw_cars(v_max, l_min, t, alpha)

KeyboardInterrupt: 

## Старый код

In [ ]:
tk = Tk()
tk.title('graph')
tk.resizable(0, 0)
canvas = Canvas(tk, width=1900, height=500, highlightthickness=0)
canvas.pack()

In [ ]:
canvas.create_line(0, 250, 1900, 250)
canvas.create_line(0, 230, 1900, 230)
canvas.create_line(0, 270, 1900, 270)
tk.update()

In [ ]:
v_max = 200 # максимальная скорость
l_min = 20 + 20 # мин расстояние до след. машины
t = 0.1 # время опроса
alpha = 0.3 # коэффициент ускорения

In [ ]:
# задать массив скоростей
arv = []
for i in range(30):
    arv.append(150)

# задать массив координат
'''
while(1):
    ar = []
    for i in range(30):
        ar.append(int(random.uniform(0, 1800)))
    ar = sorted(ar)
    f = 0
    for i in range(29):
        if ar[i + 1] - ar[i] < l_min:
            f = 1     #ЭТО НЕ РАБОТАЕТ НАДО ПОМЕНЯТЬ
    if f == 1:
        continue
    break
'''
ar = []
for i in range(30):
    ar.append((1800/30) * i)
    ar = sorted(ar)

# задать массив полосы
lines = []
for i in range(30):
    lines.append(random.randint(1, 2))

In [ ]:
cars = []
for i in range(30):
    car_ = car(canvas, ar[i], arv[i], lines[i], cars)
    cars.append(car_)
for car_ in cars:
    n_cars = car_.nearest_cars()
    car_.next_here = n_cars[0] # ссылка на следующуу машину в этом ряду
    car_.next_there = n_cars[1] # ссылка на следующую машину в соседнем ряду
    car_.prev_here = n_cars[2] 
    car_.prev_there = n_cars[3]

In [ ]:
while 1:
    for car in cars:
        car.draw()
    tk.update_idletasks()
    tk.update()
    time.sleep(0.05)

## Cломанная версия, где ездят по кругу

In [ ]:
from tkinter import *
import time
import random
import numpy as np
import math
 
tk = Tk()
tk.title('graph')
tk.resizable(0, 0)
canvas = Canvas(tk, width=1900, height=500, highlightthickness=0)
canvas.pack()
tk.update()

In [ ]:
r1 = 200
r2 = 215
r_av = (r1 + r2) / 2
lmin = 30
krug1 = canvas.create_oval(55, 55, 455, 455)
krug2 = canvas.create_oval(40, 40, 470, 470)
tk.update()

In [ ]:
class car:
    def __init__(self, canvas, phi, v, r, color, mas):
        self.canvas = canvas
        self.length = 10 # длина корпуса
        self.v = v #скорость
        self.phi = phi# координатный угол
        self.next_here = None # ссылка на следующуу машину в этом ряду
        self.next_there = None # ссылка на следующую машину в соседнем ряду
        self.prev_here = None 
        self.prev_there = None 
        self.lmin = 15 # min расстояние до следующей машины
        self.t = 0.1 # время опроса
        self.vmax = 20 # максимальная скорость
        self.alpha = 1 # коэффициент ускорения
        if r == 1:
            self.r = r1 # радиус движения
        else:
            self.r = r2
        self.xx = self.r * np.cos(self.phi) # координаты в системе координат с началом в центре холста
        self.yy = self.r * np.sin(self.phi)
        self.x = self.xx + 250 # координаты на реальном холсте
        self.y = self.yy + 250
        self.l = 0 # расстояние до следующей машины
        self.a = 0 # ускорение
        self.mas = mas # массив всех машин
        self.id = canvas.create_oval(5, 5, 5 + self.length, 5 + self.length, fill=color)
        self.canvas.move(self.id, self.x, self.y)
        
        
        # шарик узнаёт свою высоту и ширину
        self.canvas_height = self.canvas.winfo_height()
        self.canvas_width = self.canvas.winfo_width()
        
    def draw(self):
        # определяем, где предыдущая и следующая машины в соседнем ряду
        delta1 = 2 * np.pi +1
        delta2 = 2 * np.pi +1
        max_dist = 0
        for i in range(np.shape(self.mas)[0]):
            if self.mas[i].r != self.r and abs(self.mas[i].phi - self.phi) < delta1 and self.mas[i].phi < self.phi:
                self.prev_there = self.mas[i]
                delta1 = abs(self.mas[i].phi - self.phi)
            if self.mas[i].r != self.r and abs(self.mas[i].phi - self.phi) < delta2 and self.mas[i].phi > self.phi:
                self.next_there = self.mas[i]
                delta2 = abs(self.mas[i].phi - self.phi)
            if self.mas[i].r != self.r and abs(self.mas[i].phi - self.phi) > max_dist:
                tmp = i
                max_dist = abs(self.mas[i].phi - self.phi)
        if self.prev_there == None:
            self.prev_there = self.mas[tmp]
            self.mas[tmp].next_there = self
        if self.next_there == None:
            self.next_there = self.mas[tmp]
            self.mas[tmp].prev_there = self
        
        # считаем расстояние до следующей машины в своем ряду
        if ( (self.next_here.phi - self.phi) < 0):
            self.l = (self.next_here.phi - self.phi + 2 * np.pi) * r_av
        else:
            self.l = (self.next_here.phi - self.phi) * r_av
        
        # obgon        
        if ( (self.next_there.phi - self.phi) < 0):
            l_sled = (self.next_there.phi - self.phi + 2 * np.pi) * r_av
        else:
            l_sled = (self.next_there.phi - self.phi) * r_av
            
        if self.l < 35 and l_sled > 35:
            self.prev_here.next_here = self.next_here
            if self.prev_here.phi > self.prev_there.phi:
                self.prev_here.next_there = self
            if self.next_here.phi < self.next_there.phi:
                self.next_here.prev_there = self
            self.prev_there.next_here = self
            self.next_there.prev_here = self
            buf1 = self.prev_here
            buf2 = self.next_here
            self.prev_here = self.prev_there
            self.next_here = self.next_there
            self.next_there = buf2
            self.prev_there = buf1
            if self.r == r1:
                self.r = r2
            else:
                self.r = r1
        
        # еще раз считаем расстояние до следующей машины
        if ( (self.next_here.phi - self.phi) < 0):
            self.l = (self.next_here.phi - self.phi + 2 * np.pi) * r_av
        else:
            self.l = (self.next_here.phi - self.phi) * r_av
             
        la = self.l - lmin - 30
        anew = la * self.alpha
        vnew = self.v + self.a * self.t
        if self.v < 0:
            self.v = 0
        if (vnew > self.vmax):
            vnew = self.vmax
            anew = 0
        if (self.l < self.lmin + 5):
            vnew = self.next_here.v
            self.l = self.lmin + 5
            
        dphi = (self.v * self.t)/r_av + (self.a * (self.t**2))/(2 * r_av)
        if dphi < 0:
            dphi = 0
        
        self.phi = self.phi + dphi
        self.v = vnew
        self.a = anew;
        oldx = self.x
        oldy = self.y
        self.xx = self.r * np.cos(self.phi)
        self.yy = self.r * np.sin(self.phi)
        self.x = self.xx + 250
        self.y = self.yy + 250
        dx = self.x - oldx
        dy = self.y - oldy
        
        self.canvas.move(self.id, dx, dy)

In [ ]:
# нужно правильно задать и расставить по своим местам массив точек изначальный        
        
# main
ar = [] # array of phi
arv = [] # array of velocity
line = [] # array of polosa
for i in range(40):
    ar.append(random.uniform(0, 2 * np.pi))
for i in range(40):
    arv.append(20)
ar = sorted(ar)
for i in range(40):
    line.append(random.randint(1, 2))
cars = []
for i in range(40):
    cars.append(car(canvas, ar[i], arv[i], line[i], "red", cars)) 
    # на этом моменте все точки отсортированы, но не ссылаются друг на друга

# инициализируем ссылки на след и пред машины в первом ряду
for i in range(40): # находим первую машину в первом ряду
    if cars[i].r == r1:
        i_car_first_1 = i
        break
for i in reversed(range(40)): # находим последнюю машину в первом ряду
    if cars[i].r == r1:
        i_car_last_1 = i
        break
i_tmp_car = i_car_first_1
for i in range(i_car_first_1 + 1, i_car_last_1 + 1, 1):
    if cars[i].r == r1:
        cars[i_tmp_car].next_here = cars[i]
        cars[i].prev_here = cars[i_tmp_car]
        i_tmp_car = i
cars[i_car_first_1].prev_here = cars[i_car_last_1]
cars[i_car_last_1].next_here = cars[i_car_first_1]

# делаем то же во втором ряду
for i in range(40): # находим первую машину в первом ряду
    if cars[i].r == r2:
        i_car_first_2 = i
        break
for i in reversed(range(40)): # находим последнюю машину в первом ряду
    if cars[i].r == r2:
        i_car_last_2 = i
        break
i_tmp_car = i_car_first_2
for i in range(i_car_first_2 + 1, i_car_last_2 + 1, 1):
    if cars[i].r == r2:
        cars[i_tmp_car].next_here = cars[i]
        cars[i].prev_here = cars[i_tmp_car]
        i_tmp_car = i
cars[i_car_first_2].prev_here = cars[i_car_last_2]
cars[i_car_last_2].next_here = cars[i_car_first_2]        

In [ ]:
for i in range(40):
    if cars[i].next_here == None or cars[i].prev_here == None:
        print(i)

In [ ]:
while 1:
    for i in range(20):
        cars[i].draw()
    tk.update_idletasks()
    tk.update()
    time.sleep(0.02)